# Finding the cap rate of each row

In [1]:
import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import math

## load models

In [2]:
sales_model = joblib.load('../models/sales_predict.pkl')
rent_model = joblib.load('../models/rent_predict.pkl')

## load dataset

In [3]:
df = pd.read_csv('../tables/sales.csv')

## preprocces

In [4]:
df = df.drop(columns=['neighborhood'])
df = df.dropna()
df = df.drop_duplicates()
not_to_small = df["house_area"] > 15
not_to_big = df["house_area"] < 500
garden_not_to_big = df["garden_area"] < 1000
garden_not_to_small = df["garden_area"] >= 0
furniture = df["furniture"] < 2

df = df.where(not_to_small & not_to_big & garden_not_to_big & garden_not_to_small & furniture).dropna()
not_to_cheap = df["price"] > 200000
not_to_expensive= df["price"] < 15000000
df = df.where(not_to_cheap & not_to_expensive).dropna()
not_to_little_rooms = df["rooms"] >= 2
not_to_many_rooms = df["rooms"] <= 10
df = df.where ( not_to_little_rooms & not_to_little_rooms ).dropna()
df['house type'] = df['house type'].str.replace('&nbsp;', '')
df['house type'] = df['house type'].str.replace('/קוטג', '')
df['house type'] = df['house type'].str.replace("'", '')

types_to_keep = ['דירה', 'בית פרטי', 'דירת גן', 'גג/פנטהאוז', 'דו משפחתי', 'דופלקס', 'פרטי']
df = df[df['house type'].isin(types_to_keep)]

list_to_replace = ['דירה', 'בית פרטי', 'דירת גן', 'גג/פנטהאוז', 'דו משפחתי', 'דופלקס', 'פרטי']
replce_with = ['apartment', 'private house', 'garden apartment', 'penthouse', 'dual family', 'duplex', 'private house' ]
df = df.replace(to_replace =list_to_replace, value =replce_with)

city_counts = df['city'].value_counts()
top_10_cities = city_counts.head(10)
top_cities_list = top_10_cities.index.tolist()
df = df[df['city'].isin(top_cities_list)]
list_to_replace = ['תל אביב יפו', 'ירושלים', 'חיפה', 'באר שבע', 'רמת גן', 'נתניה', 'פתח תקווה', 'ראשון לציון', 'אשדוד', 'אשקלון']
replce_with = ['tel aviv', 'jerusalem', 'haifa', 'beer sheva', 'ramat gan', 'natanya', 'petah tikva', 'rishon lezion', 'ashdod', 'ashkelon']
df = df.replace(to_replace =list_to_replace, value =replce_with)

le1 = LabelEncoder()
le2 = LabelEncoder()
df['city'] = le1.fit_transform(df['city'])
df['house type'] = le2.fit_transform(df['house type'])

X = df.drop('price', axis=1)
y = np.log(df['price'])

### predict the rent price and cap rate, add to the new csv file

In [5]:
rent_price = []
cap_rate = []

In [6]:
def predict_rows(model, row_index):
    selected_row = X.iloc[row_index]
    temp_df = pd.DataFrame(selected_row).T
    temp_df = temp_df.reset_index(drop=True)
    rent_predicted_price = model.predict(temp_df)
    rent_predicted_price = round(math.exp(rent_predicted_price[0]))
    sale_price = df['price'].iloc[row_index]
    new_cap_rate = (rent_predicted_price * 12 * 100)/sale_price
    new_cap_rate = '{:.3f}'.format(round(new_cap_rate, 5))
    rent_price.append(rent_predicted_price)
    cap_rate.append(new_cap_rate)

In [7]:
from tqdm import tqdm
for i in tqdm(range(len(X))):
    predict_rows(rent_model, i)

100%|██████████| 35974/35974 [04:31<00:00, 132.54it/s]


In [8]:
final_csv = df.copy()
final_csv['rent price'] = rent_price
final_csv['cap rate'] = cap_rate

In [9]:
final_csv['city'] = le1.inverse_transform(final_csv['city'])
final_csv['house type'] = le2.inverse_transform(final_csv['house type'])
print(final_csv.head())

         city house type  house_area  garden_area  rooms  balconies  \
0    tel aviv  apartment        85.0          0.0    4.0        0.0   
2    tel aviv  apartment       102.0          0.0    4.0        1.0   
3  beer sheva  apartment        60.0          0.0    3.5        1.0   
4  beer sheva  apartment       202.0          0.0    5.0        2.0   
6   ramat gan  apartment        87.0          0.0    3.0        3.0   

   air_condition  parking  protected_room  elevator  renovated  furniture  \
0            1.0      0.0             1.0       1.0        1.0        0.0   
2            1.0      0.0             0.0       1.0        0.0        1.0   
3            1.0      1.0             0.0       0.0        1.0        0.0   
4            1.0      1.0             1.0       1.0        1.0        0.0   
6            1.0      0.0             0.0       0.0        1.0        0.0   

   accessibility  bars  storage      price  rent price cap rate  
0            1.0   0.0      0.0  4690000.0  

In [10]:
final_csv.to_csv('../tables/final_csv.csv', index=False)

In [11]:
cities_encoder = {}
for name in replce_with:
    cities_encoder[name] = le1.transform([name])[0]

In [13]:
house_types = ['apartment', 'private house', 'garden apartment', 'dual family', 'duplex', 'private house', 'penthouse']
house_type_encoder = {}
for name in house_types:
    house_type_encoder[name] = le2.transform([name])[0]

In [14]:
print(house_type_encoder)

{'apartment': 0, 'private house': 5, 'garden apartment': 3, 'dual family': 1, 'duplex': 2, 'penthouse': 4}
